In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('./Bilan tarification 2019.csv', sep=";", encoding="iso-8859-1")
# df

In [3]:
df

,Id Personne,Id Famille,QF,Type repas,Nombre,id gale ou non,type de repas simplifi par tarif identique
0,1,325,999999,Repas Végétarien,2,NaN,Repas vgtarien
1,2,497,636,Repas Standard,1,NaN,Repas Standard
2,3,762,999999,Repas Standard,1,NaN,Repas Standard
3,4,770,392,Repas Standard,125,oui,Repas Standard
4,4,770,999999,Repas Standard,1,NaN,Repas Standard
...,...,...,...,...,...,...,...
16930,15579,10849,999999,Repas Sans Porc,8,NaN,Repas Standard
16931,15580,10850,999999,Repas Standard,13,NaN,Repas Standard
16932,15581,10851,259,Repas Standard,12,NaN,Repas Standard
16933,15582,10852,620,Repas Standard,14,NaN,Repas Standard


In [4]:
df['Repas'] = np.select(
    [
        df["Type repas"].str.contains('tarien'), #tarien = végétarien
        df["Type repas"].str.contains('Panier'),
    ],
    [
        'cantine_vegetarien',
        'cantine_panier',
    ],
    default = 'cantine'
)
# df

In [5]:
df.QF = np.minimum(df.QF, 99999)

In [6]:
raw_df = pd.pivot_table(df[0:150], index=["Id Personne", "Id Famille", "QF"], columns="Repas", values="Nombre", fill_value=0, aggfunc=np.sum)
raw_df

Repas                         cantine  cantine_panier  cantine_vegetarien
Id Personne Id Famille QF                                                
1           325        99999        0               0                   2
2           497        636          1               0                   0
3           762        99999        1               0                   0
4           770        392        125               0                   0
                       99999        1               0                   0
...                               ...             ...                 ...
137         9122       99999       19               0                   0
138         1730       4130       127               0                   0
139         1352       506         34               0                   0
140         1735       1238        98               0                   0
141         1736       1582       128               0                   0

[147 rows x 3 columns]

In [7]:
flat_df = raw_df.reset_index()

In [8]:
familles_ids = flat_df["Id Famille"] * 100000 + flat_df.QF
# familles_ids

In [9]:
(unique_famille_ids, famille_idx, famille_id_inv) = np.unique(familles_ids, return_index=True, return_inverse=True)
# unique_famille_ids, famille_idx, famille_id_inv

In [10]:
individu_df = pd.DataFrame({
    'famille_id': famille_id_inv,
    'strasbourg_metropole_nombre_repas_cantine':flat_df.cantine,
    'strasbourg_metropole_nombre_repas_cantine_vegetarien':flat_df.cantine_vegetarien,
    'strasbourg_metropole_nombre_repas_cantine_panier':flat_df.cantine_panier,
})
# individu_df # nombre de repas pour chaque enfant selon le type de repas 

In [11]:
famille_df = pd.DataFrame({
    'strasbourg_metropole_quotient_familial': flat_df.QF[famille_idx]
})
# famille_df

In [12]:
base_year = "2021-03"

In [13]:
from openfisca_survey_manager.scenarios import AbstractSurveyScenario
from openfisca_france import CountryTaxBenefitSystem
from openfisca_france.model.base import Famille, FoyerFiscal, Menage
from openfisca_core import periods, parameters
from openfisca_core.model_api import Reform

In [14]:
individu_df['famille_role_index'] = 0
individu_df['foyer_fiscal_id'] = individu_df.famille_id
individu_df['foyer_fiscal_role_index'] = 0
individu_df['menage_id'] = individu_df.famille_id
individu_df['menage_role_index'] = 0

In [15]:
menage_df = pd.DataFrame({})
foyerfiscaux_df = pd.DataFrame({})

In [16]:
data = dict(input_data_frame_by_entity_by_period = {periods.period(base_year): dict(
individu=individu_df,
famille=famille_df,
menage=menage_df,
foyer_fiscal=foyerfiscaux_df)})

In [17]:
class StrasbourgSurveyScenario(AbstractSurveyScenario):
    def __init__(self, data = None, reform = None):
        super(StrasbourgSurveyScenario, self).__init__()

        base = CountryTaxBenefitSystem()
        base.load_extension('openfisca_france_local')
        if reform is not None:
            tax_benefit_system = reform(base)
        else:
            tax_benefit_system = base
        self.year = base_year

        if 'input_data_frame_by_entity_by_period' in data:
            period = periods.period(self.year)
            dataframe_variables = set()
            for entity_dataframe in data['input_data_frame_by_entity_by_period'][period].values():
                if not isinstance(entity_dataframe, pd.DataFrame):
                    continue
                dataframe_variables = dataframe_variables.union(set(entity_dataframe.columns))
            self.used_as_input_variables = list(
                set(tax_benefit_system.variables.keys()).intersection(dataframe_variables)
                )

        self.set_tax_benefit_systems(tax_benefit_system)
        self.init_from_data(data = data)

In [18]:
def compute_result(data, reform):
    scenario = StrasbourgSurveyScenario(data = data, reform = reform)
    cout_cantine = scenario.simulation.calculate('strasbourg_metropole_cout_cantine', period = '2021-03')
    cout_cantine_vege = scenario.simulation.calculate('strasbourg_metropole_cout_cantine_repas_vegetarien', period = '2021-03')
    cout_cantine_panier = scenario.simulation.calculate('strasbourg_metropole_cout_cantine_repas_panier', period = '2021-03')
    aide_etat = scenario.simulation.calculate('strasbourg_metropole_aide_repas_etat', period = '2021-03')
    return (scenario, np.array([sum(cout_cantine), sum(cout_cantine_vege), sum(cout_cantine_panier)]), np.array([sum(aide_etat)]))

In [19]:
base = compute_result(data, None)

In [20]:
base # somme du nombre de repas en fonction du type de repas 

(<__main__.StrasbourgSurveyScenario at 0x7f76dd87e690>,
 array([52072.25109744,  2674.79992104,   180.90000916]),
 array([0.]))

In [21]:
def modify_parameters(local_parameters):
    cantine = []
    vege = []
    panier = []

    valeurs = [ # QF, tarif cantine, végétarien, panier
        [-1, 1.5, 1.15, 0.75],
        [410, 2.15, 1.80, 1.08],
        [510, 2.70, 2.35, 1.35],
        [620, 3.25, 2.90, 1.63],
        [720, 3.80, 3.45, 1.90],
        [820, 4.40, 4.05, 2.20],
        [920, 5.10, 4.75, 2.55],
        [1030, 5.80, 5.45, 2.90],
        [1540, 6.30, 5.95, 3.15],
        [2050, 6.90, 6.55, 3.45]
    ]

    for tranche in valeurs:
        (qf, c, v, p) = tranche
        cantine.append(parameters.ParameterScaleBracket(data={
            'threshold': { '2010-10-01': { 'value': qf } },
            'amount': { '2010-10-01': { 'value': c } }
        }))
        vege.append(parameters.ParameterScaleBracket(data={
            'threshold': { '2010-10-01': { 'value': qf } },
            'amount': { '2010-10-01': { 'value': v } }
        }))
        panier.append(parameters.ParameterScaleBracket(data={
            'threshold': { '2010-10-01': { 'value': qf } },
            'amount': { '2010-10-01': { 'value': p } }
        }))

    local_parameters.metropoles.strasbourg.tarifs_cantine.brackets = cantine
    local_parameters.metropoles.strasbourg.tarifs_repas_vege.brackets = vege
    local_parameters.metropoles.strasbourg.tarifs_repas_panier.brackets = panier
    return local_parameters
    
class simulation(Reform):
    name = u"Fusionne les trois premières tranches"

    def apply(self):
        self.modify_parameters(modifier_function = modify_parameters)

In [22]:
result_simulation = compute_result(data, simulation)

In [23]:
result_simulation

(<__main__.StrasbourgSurveyScenario at 0x7f76dc157250>,
 array([52072.25109744,  2674.79992104,   180.90000916]),
 array([0.]))